##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Sous licence Apache, Version 2.0 (la "Licence");
# vous ne pouvez pas utiliser ce fichier, sauf conformément avec la licence.
# Vous pouvez obtenir une copie de la Licence sur
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Sauf si requis par la loi en vigueur ou par accord écrit, le logiciel
# distribué sous la licence est distribué "TEL QUEL", SANS GARANTIE NI
# CONDITION DE QUELQUE NATURE QUE CE SOIT, implicite ou explicite.
# Consultez la Licence pour connaître la terminologie spécifique régissant
# les autorisations et les limites prévues par la Licence.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# L'autorisation est accordée, gracieusement, à toute personne acquérant 
# une copie de cette bibliothèque et des fichiers de documentation associés (la "Bibliothèque"),
# de commercialiser la Bibliothèque sans restriction, notamment les droits d'utiliser,
# de copier, de modifier, de fusionner, de publier, de distribuer, de sous-licencier 
# et/ou de vendre des copies de la Bibliothèque, ainsi que d'autoriser les personnes 
# auxquelles la Bibliothèque est fournie à le faire, sous réserve des conditions suivantes :
#
# La déclaration de copyright ci-dessus et la présente autorisation doivent être 
# incluses dans toutes copies ou parties substantielles de la Bibliothèque.
#
# LA BIBLIOTHÈQUE EST FOURNIE "TELLE QUELLE", SANS GARANTIE D'AUCUNE SORTE, EXPLICITE OU 
# IMPLICITE, NOTAMMENT SANS GARANTIE DE QUALITÉ MARCHANDE, D’ADÉQUATION À UN USAGE PARTICULIER
# ET D'ABSENCE DE CONTREFAÇON. EN AUCUN CAS, LES AUTEURS OU TITULAIRES DU DROIT D'AUTEUR 
# NE SERONT RESPONSABLES DE TOUT DOMMAGE, RÉCLAMATION OU AUTRE RESPONSABILITÉ,
# QUE CE SOIT DANS LE CADRE D'UN CONTRAT, D'UN DÉLIT OU AUTRE, EN PROVENANCE DE,
# CONSÉCUTIF À OU EN RELATION AVEC LA BIBLIOTHÈQUE OU SON UTILISATION, OU AVEC 
# D'AUTRES ÉLÉMENTS DE LA BIBLIOTHÈQUE.

# Classification de Base : Classer des images de vetêments

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Voir sur TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/fr/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Exécuter dans Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/fr/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Voir la source sur GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/fr/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Voir la source sur notebook</a>
  </td>
</table>

Ce guide forme un modèle de réseau neuronal pour classer les images de vêtements, comme les baskets et les chemises. C'est normal si vous ne comprenez pas tous les détails; il s'agit d'un aperçu rapide d'un programme TensorFlow complet avec les détails expliqués au fur et à mesure.

Ce guide utilise [tf.keras](https://www.tensorflow.org/guide/keras), une API de haut niveau pour construire et former des modèles dans TensorFlow.


In [0]:
try:
  # %tensorflow_version existe uniquement dans Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow et tf.keras
import tensorflow as tf
from tensorflow import keras

# Libraries d'aide
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Importer le jeu de données Fashion MNIST

Ce guide utilise l'ensemble de données [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) qui contient 70 000 images en niveaux de gris dans 10 catégories. Les images montrent des vêtements individuels à basse résolution (28 x 28 pixels), comme ici :

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Échantillons Fashion-MNIST</a> (par Zalando, License MIT).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST est destiné à remplacer l'ensemble de données [MNIST](http://yann.lecun.com/exdb/mnist/) classique, souvent utilisé comme le "Hello, World" des programmes d'apprentissage automatique de vision par ordinateur. Le jeu de données MNIST contient des images de chiffres manuscrits (0, 1, 2, etc.) dans un format identique à celui des vêtements que vous allez utiliser ici.


Ce guide utilise Fashion MNIST pour la variété, et parce que c'est un problème un peu plus difficile que le MNIST régulier. Les deux ensembles de données sont relativement petits et sont utilisés pour vérifier qu'un algorithme fonctionne comme prévu. Ce sont de bons points de départ pour tester et déboguer le code.

Ici, 60 000 images sont utilisées pour former le réseau et 10 000 images pour évaluer la précision avec laquelle le réseau a appris à classer les images. Vous pouvez accéder au Fashion MNIST directement depuis TensorFlow. Importez et chargez les données Fashion MNIST directement depuis TensorFlow :

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Le chargement du jeu de données retourne 4 tableaux NumPy:

* Les tableaux `train_images` et `train_labels` sont les *ensembles d'entraînement* - les données que le modèle utilise pour apprendre.
* Le modèle est testé par rapport aux tableaux *test set*, `test_images` et `test_labels`.

Les images sont des tableaux NumPy 28x28, avec des valeurs de pixels comprises entre 0 et 255. Les *labels* (étiquettes) sont un tableau d'entiers, allant de 0 à 9. Ceux-ci correspondent à la *class* (classe) de vêtements que l'image représente :

<table>
  <tr>
    <th>Etiquettes</th>
    <th>Classes</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/haut</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Pantalon</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pull</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Robe</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Manteau</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandale</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Chemise</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Chaussure de sport</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Sac</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Botte haute</td>
  </tr>
</table>

Chaque image est mappée à une seule étiquette. Comme les *noms de classe* ne sont pas inclus dans l'ensemble de données, stockez-les ici pour les utiliser plus tard lors de la préparation des images :

In [0]:
class_names = ['T-shirt/haut', 'Pantalon', 'Pull', 'Robe', 
               'Manteau', 'Sandale', 'Chemise', 
               'Chaussure de sport', 'Sac', 'Botte haute']

## Examinons les données

Examinons le format de ce jeu de données avant de former le modèle. Ce qui suit montre qu'il y a 60 000 images dans le kit d'entraînement, chaque image étant représentée en 28 x 28 pixels :

In [0]:
train_images.shape

Dans le kit de formation, il y a 60 000 étiquettes :

In [0]:
len(train_labels)

Chaque étiquette est un entier compris entre 0 et 9 :

In [0]:
train_labels

Il y a 10 000 images dans l'ensemble de test. Comme précédemment, chaque image est représentée en 28 x 28 pixels :

In [0]:
test_images.shape

Et l'ensemble de test contient 10 000 étiquettes d'images :

In [0]:
len(test_labels)

## Prétraitement des données

Les données doivent être prétraitées avant la formation du réseau. Si vous inspectez la première image du jeu d'apprentissage, vous verrez que les valeurs de pixels se situent entre 0 et 255 :

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Mettez ces valeurs à l'échelle sur une plage de 0 à 1 avant de les transmettre au modèle de réseau neuronal. Pour ce faire, divisez les valeurs par 255. Il est important que l'*ensemble de formation* et l'*ensemble de test* soient prétraités de la même manière :

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Pour vérifier que les données sont dans le bon format et que vous êtes prêt à construire et former le réseau, affichons les 25 premières images de l'*ensemble de formation* et le nom de la classe sous chaque image.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Construire le modèle

La construction du réseau neuronal nécessite la configuration des couches du modèle, puis la compilation du modèle.

### Préparer les couches

L'élément de base d'un réseau neuronal est la *couche*. Les calques extraient les représentations des données qui leur sont fournies. On espère que ces représentations soient significatives pour le problème en question.

La plus grande partie de l'apprentissage profond consiste à enchaîner des couches simples. La plupart des couches, telles que `tf.keras.layers.Dense`, ont des paramètres qui sont appris pendant l'entraînement.

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

La première couche de ce réseau, `tf.keras.layers.Flatten`, transforme le format des images d'un tableau bidimensionnel (de 28 x 28 pixels) en un tableau unidimensionnel (de 28 * 28 = 784 pixels). Pensez à cette couche comme à décomposition en ligne de pixels de l'image et un alignement de celles-ci. Cette couche n'a pas de paramètres à apprendre; elle ne fait que reformater les données.

Une fois les pixels aplatis, le réseau se compose d'une séquence de deux couches `tf.keras.layers.Dense`. Il s'agit de couches neuronales fortement connectées ou entièrement connectées. La première couche `Dense` a 128 noeuds (ou neurones). La deuxième (et dernière) couche est une couche de 10 nœuds *softmax* qui retourne un tableau de 10 scores de probabilité qui totalisent 1. Chaque nœud contient un score qui indique la probabilité que l'image courante appartienne à l'une des 10 classes.

### Compiler le modèle

Avant que le modèle ne soit prêt pour la formation, il a besoin de quelques réglages supplémentaires. Ceux-ci sont ajoutés lors de l'étape *compilation* du modèle :

* *Fonction de perte* — Cela permet de mesurer la précision du modèle pendant la formation. Vous voulez minimiser cette fonction pour "diriger" le modèle dans la bonne direction.
* *Optimiseur* — De cette manière, le modèle est mis à jour en fonction des données observées et de sa fonction de perte..
* *Métriques* — Utilisé pour surveiller les étapes de formation et de test. L'exemple suivant utilise l'*exactitude*, la fraction des images qui sont correctement classées.

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Former le modèle

La formation du modèle de réseau neuronal nécessite les étapes suivantes :

1. Transmettre les données de formation au modèle. Dans cet exemple, les données d'entraînement se trouvent dans les tableaux `train_images` et `train_labels`.
2. Le modèle apprend à associer les images et étiquettes.
3. Vous demandez au modèle de faire des prédictions sur un ensemble de test dans cet exemple, le tableau `test_images`. Vérifiez que les prédictions correspondent aux étiquettes du tableau `test_labels`.

Pour commencer la formation, appelez la méthode `model.fit` — elle appelée comme cela parce qu'elle "adapte" le modèle aux données de formation :

In [0]:
model.fit(train_images, train_labels, epochs=10)

Au fur et à mesure que le modèle s'entraîne, les métriques de perte et de précision sont affichées. Ce modèle atteint une précision d'environ 0,88 (ou 88 %) sur les données de formation.

## Évaluer l'exactitude

Ensuite, comparez les performances du modèle sur l'ensemble de données de test :

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nPrécision du test:', test_acc)

Il s'avère que la précision de l'ensemble de données de test est un peu inférieure à celle de l'ensemble de données de formation. Cet écart entre la précision de l'entraînement et la précision du test représente un *débordement*. Un modèle d'apprentissage machine est surajusté lorsqu'il donne de moins bons résultats sur de nouvelles données d'entrée que sur les données de formation.

## Faire des prédictions

Une fois le modèle formé, vous pouvez l'utiliser pour faire des prédictions sur certaines images.

In [0]:
predictions = model.predict(test_images)

Ici, le modèle a prédit l'étiquette pour chaque image de l'ensemble de test. Jetons un coup d'oeil à la première prédiction :

In [0]:
predictions[0]

Une prédiction est un tableau de 10 nombres. Ils représentent la "confiance" du modèle dans le fait que l'image correspond à chacun des 10 vêtements différents. Vous pouvez voir quelle étiquette a la valeur de confiance la plus élevée :

In [0]:
np.argmax(predictions[0])

Ainsi, le modèle est plus confiant que cette image est une botte haute, ou `class_names[9]`. L'examen de l'étiquette d'essai montre que cette classification est correcte :

In [0]:
test_labels[0]

Tracez un graphique pour voir l'ensemble complet des 10 prédictions de classe.

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Regardons la première image, les prédictions et le tableau des prédictions. Les étiquettes de prédiction correctes sont bleues et les étiquettes de prédiction incorrectes sont rouges. Le nombre donne le pourcentage (sur 100) pour l'étiquette prévue.

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Représentons plusieurs images avec leurs prédictions. Notez que le modèle peut se tromper même s'il est très confiant.

In [0]:
# Représentez les premières X images du test, leurs étiquettes prévues et les véritables étiquettes.
# Les prédictions correctes sont en bleu et les prédictions incorrectes en rouge.
Les prédictions correctes sont en bleu et les prédictions incorrectes en rouge.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

Enfin, utilisez le modèle formé pour faire une prédiction sur une seule image.

In [0]:
# Récupérer une image du jeu de données de test.
img = test_images[1]

print(img.shape)

Les modèles `tf.keras` sont optimisés pour faire des prédictions sur un *lot*, ou une collection, d'exemples tout à la fois. Par conséquent, même si vous utilisez une seule image, vous devez l'ajouter à une liste :

In [0]:
# Ajoutez l'image à un lot dont il est le seul membre.
img = (np.expand_dims(img,0))

print(img.shape)

Prévoyez maintenant l'étiquette correcte pour cette image :

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`model.predict` renvoie une liste de listes — une liste pour chaque image du lot de données. Récupérez les prédictions pour notre (seule) image dans le lot :

In [0]:
np.argmax(predictions_single[0])

Et le modèle prédit une étiquette comme prévu.

Note: Notre communauté `TensorFlow` a traduit ces documents. 
Étant donné que les traductions effectuées dans la communauté sont plus optimales, 
rien ne garantit qu'il s'agit d'un reflet exact et à jour de la documentation anglaise officielle. 
Si vous avez des suggestions pour améliorer cette traduction, 
veuillez envoyer une demande d'extraction ou `pull request`) à [tensorflow/docs](https://github.com/tensorflow/docs). 
Pour vous porter volontaire pour écrire ou réviser les traductions de la communauté, 
contactez la [liste docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).
